In [1]:
import torch
from torch.utils.data import DataLoader, DistributedSampler
import transformers
from transformers import AutoConfig, AutoTokenizer, HfArgumentParser
from sklearn.metrics import average_precision_score
import numpy as np
import sys
sys.path.append("/home/jovyan/filtered-transformer/")

from examples.spliceai.data import SpliceAIDataset

tokenizer = AutoTokenizer.from_pretrained("/home/jovyan/filtered-transformer/data/tokenizers/t2t_1000h_multi_32k/")
data_path = "/home/jovyan/splice/train.csv.gz"
train_dataset = SpliceAIDataset(data_path, tokenizer, max_seq_len=3024, targets_offset=5000, targets_len=5000)




2023-03-22 02:11:44.421517: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 02:11:44.606446: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-22 02:11:46.537969: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-22 02:11:46.538074: W tensorflow/

In [2]:
train_dataset[0]


{'input_ids': array([   1, 6588, 2662, ...,    3,    3,    2]),
 'token_type_ids': array([0, 0, 0, ..., 0, 0, 0]),
 'attention_mask': array([1, 1, 1, ..., 0, 0, 1]),
 'labels': array([[-100, -100],
        [-100, -100],
        [-100, -100],
        ...,
        [-100, -100],
        [-100, -100],
        [-100, -100]]),
 'labels_ohe': array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 'labels_mask': array([0., 0., 0., ..., 0., 0., 0.])}

In [2]:
from typing import Dict
from torch import Tensor
from data_filters.sliding_window import SlidingWindowFilterDict
from memup.base import MemoryRollout
from examples.spliceai.modules import MemUpMemoryImpl, BertForSpliceAI, DataCollectorTrain
from memup.preproc import IncrementStep
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

data_filter = SlidingWindowFilterDict(512, pad_fields={}, padding=0, skip_fields={"length"})

model_cfg = AutoConfig.from_pretrained('/home/jovyan/filtered-transformer/data/configs/L12-H768-A12-V32k-preln.json')
model = BertForSpliceAI(config=model_cfg)

memup_iter = MemoryRollout[Dict[str, torch.Tensor]](
    steps=2,
    memory=MemUpMemoryImpl(model),
    data_filter=data_filter,
    info_update=[IncrementStep()]
)



In [3]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=32, num_workers=8)

for it, batch in enumerate(train_dataloader):

        model.train()
        
        info = {}
        done = False
        print()
        state = torch.zeros(batch["labels"].shape[0], 100, model_cfg.hidden_size, device=torch.device("cuda:0"))
        batch["length"] = batch["labels"].shape[1]

        while not done:

                data_collector, state, info, done = memup_iter.forward(batch, state, info, DataCollectorTrain())
                print(it, info)


TypeError: BertForSpliceAI.forward() got an unexpected keyword argument 'labels_ohe'